## 1. Importar librerías

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

## 2. Importar el dataset y analizar los datos

In [2]:
data = pd.read_csv('data/car-sales-extended-missing-data.csv')
data.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


Tenemos 5 columnas:
1. Make - Marca del coche
2. Colour
3. Kilómetros
4. Número de puertas
5. Precio - es nuestro obejtivo a predecir

In [3]:
data.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [4]:
data.nunique()

Make               4
Colour             5
Odometer (KM)    948
Doors              3
Price            934
dtype: int64

## 3. Depurar los datos (imputer + encoder + column transform)

Estrategia:
1. Eliminar filas sin precio
2. Convertir Make, Colour y Price en categorías
3. Sustituir valores desconocidos por una nueva categoría, excepto en el caso de las puertas, en ese caso usamos la moda
4. Utilizar el valor medio para los km

In [5]:
print(data.shape)
data = data.dropna(subset=['Price'])
print(data.shape)

(1000, 5)
(950, 5)


In [35]:
# Tipos de datos
categorical_features = ['Make', 'Colour']
numerical_features = ['Odometer (KM)']
doors_features = ['Doors']

# Pipelines para transformar los datos. 1. Sustituir valore 2. Convertir en categorias si corresponde
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'))
])

door_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=4)),
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

In [41]:
# Preparamos la transformación de los datos
column_transformer = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_features),
    ('door', door_transformer, doors_features),
    ('numerical', numerical_transformer, numerical_features)
])

## 4. Dividir los datos y entrenar al modelo

In [56]:
# Dividir los datos
X = data.drop(columns=['Price'])
y = data['Price']
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [57]:
# Crear el modelo
model = RandomForestRegressor() # Dejamos los valores por defecto

In [58]:
# Cerar un pipeline para preprocesar los datos y entrenar al modelo
model_pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('train', model)
])

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Colour']),
                                                 ('door',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=4,
                                                                                 strategy='constant')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Doors']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Odometer (KM)'])])),
                ('train', RandomForestRegressor())])

In [59]:
# Crear una pipline para evaluar el modelo
model_pipeline.score(X_test, y_test)

0.2254286556708368